In [ ]:
# To be able to close the plots
%matplotlib inline

from ipywidgets import Button, Layout

root_dir = "saved_results"

save_eval_btn = Button(description="Save evaluation results", layout=Layout(width='20%'))
save_seperatly_btn = Button(description="Save results in sub-folder", layout=Layout(width='20%'))

display(save_eval_btn, save_seperatly_btn)

def safe_chmod(path: str, mode: int):
    try:
        os.chmod(path, mode)
    except:
        pass

def save_evaluation_results(btn):
    global eval_results
    btn.disabled = True
    btn.description = "Saving.."
    result_file = f"{root_dir}/Results.csv"
    all_eval_results = pd.DataFrame()
    for it in eval_results:
        all_eval_results = all_eval_results.append(it)
    if os.path.isfile(result_file):
        unique_cols = list(all_eval_results.columns)
        unique_cols.remove("Date")
        full_results = pd.read_csv(result_file, sep=';', index_col=0)
        full_results = full_results.append(all_eval_results)
        full_results.drop_duplicates(unique_cols, 'last', True)
    else:
        full_results = all_eval_results
    full_results.to_csv(result_file, sep=';')
    safe_chmod(result_file, 0o664)
    btn.description = "Results saved"


def save_seperatly_clicked(btn):
    global eval_results, eval_plots, dfs_with_gaps, imputed_dfs, dfloader, imputer
    btn.disabled = True
    btn.description = "Saving in sub-folder..."
    for target, results, plots in zip(dfloader.targets, eval_results, eval_plots):
        dirname = f"{root_dir}/{imputer['title']} - {target} - {imputation_date}"
        result_file = f"{dirname}/Results.csv"
        os.makedirs(dirname, mode = 0o774, exist_ok=True)
        results.to_csv(result_file)
        safe_chmod(result_file, 0o664)
        for i in range(len(imputed_dfs)):
            figfile = f"{dirname}/Gap type {i + 1}.png"
            plots[i].savefig(figfile, dpi=300)
            safe_chmod(figfile, 0o664)
            df = pd.DataFrame({
                    f"Gapped {target}": dfs_with_gaps[i][target].values,
                    f"Imputed {target}": imputed_dfs[i][target].values
                }, index=imputed_dfs[i].index)
            dffile = f"{dirname}/Gap type {i + 1}.csv"
            df.to_csv(dffile, sep=';')
            safe_chmod(dffile, 0o664)
    btn.description = "Results saved in sub-folder"


save_eval_btn.on_click(save_evaluation_results)
save_seperatly_btn.on_click(save_seperatly_clicked)